<a name='1'></a>
## Import Packages and load data

In [1]:
### -*- coding: utf-8 -*-
"""
Created on Mon May 24 13:26:13 2021

@author: kjsanche

Description: 
A function to load the 5 minute granules from MODIS channel 1 
(0.65 microns) and the contrail mask for ML with a CNN.

To do:
ASAP:
-separate testing data
-plot testing data vs mask



lower priority:
-optimize image shape by transforming back to original satellite swath projection
-save and set up to load data as tfrecord
-organize/markdown/comment code


Input:
Path   (string)

        
        
Output:
MODISCh1 (2D uint32)
MASK     (2D uint16)
"""


from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib
import numpy as np
import struct
import os
import glob
import collections
import cv2
import gc
import json
import random
from UNET_Functions import unet_model, summary
from Sat_contrail_read import Extract_RawDef, extract_img, extract_mask, extract_imglist2, get_model_memory_usage
from scipy.spatial import ConvexHull
from scipy.ndimage import rotate, measurements
from format_input import write_images_to_tfr_long# *
from operator import itemgetter
import tensorflow as tf
from tensorflow.keras import layers
import random
from PIL import Image
from scipy import signal
import skimage.exposure
from sklearn import linear_model
from typing import Any, Dict, Iterable, List, Mapping, Optional, Tuple

sys_details = tf.sysconfig.get_build_info()
print(sys_details)
cudnn_version = sys_details["cudnn_version"]
cuda_version = sys_details["cuda_version"]

print('cuda version: ', cuda_version)
print('cudNN version: ',cudnn_version)
print('TF version: ', tf.version.VERSION)


############# dataset settings ######################
DVERSION = 'v28' #iterate version number to keep track in spreadsheet
NCHANNELS=3 #number of channels + 1 (3, 7 or 8)
img_slice_size = 64 #pixel width/height per image slice
TRAIN_SPLIT = 0.8 #portion of data used for training
AUGMENT_SCALING = 8
contrail_ratio = 500 *AUGMENT_SCALING #number of slices to keep without contrails:slices with contrails (2 = 2:1), *8 needed because augmentation boost number of contrail images by a factor of 8
pixel_threshold = 10#(img_slice_size**2)*0.001 #pixel threshold to determine if slice has a significant number of pixels labeled as contrail

############### dataset type #######################
#if both AUGMENT and BALANCE are False, the validation or test set will be created
AUGMENT = True
AUGMENT_SHIFT = True
BALANCE = True
# CREATE_VALIDATION_SET only matters if AUGMENT and BALANCE are both false
CREATE_VALIDATION_SET = True  #if false create test set
####################################################


#tf.compat.v1.disable_eager_execution()
#tf.compat.v1.experimental.output_all_intermediates(True) 
path = os.getcwd()
proj_path = os.path.normpath(path + os.sep + os.pardir) #get parent directory

save_TFrecord_path = '/home/kjsanche/Desktop/TFrecords/' 
label_path = '/home/kjsanche/Desktop/ExternalSSD/SatContrailData/landsat8/labels/'
image_path = '/home/kjsanche/Desktop/ExternalSSD/SatContrailData/landsat8/images/'

all_label_files = glob.glob(label_path + "landsat_contrails*", recursive = True)
all_label_files = np.sort(all_label_files)

2022-07-13 12:11:32.560679: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


OrderedDict([('cpu_compiler', '/home/builder/ktietz/aggregate/tensorflow_recipes/ci_cpu/tensorflow-base_1614583966145/_build_env/bin/x86_64-conda_cos6-linux-gnu-gcc'), ('cuda_compute_capabilities', ['compute_35', 'compute_52', 'compute_60', 'compute_61', 'compute_70', 'compute_75']), ('cuda_version', '10.1'), ('cudnn_version', '7'), ('is_cuda_build', True), ('is_rocm_build', False)])
cuda version:  10.1
cudNN version:  7
TF version:  2.4.1


## download landsat8 images corresponding to labeled cases

In [2]:
data_split = range(0,int(len(all_label_files)*TRAIN_SPLIT)) #train dataset
if not AUGMENT and not BALANCE:
    if CREATE_VALIDATION_SET: #validation dataset
        data_split = range(int(len(all_label_files)*TRAIN_SPLIT),int(len(all_label_files)*(TRAIN_SPLIT+(1-TRAIN_SPLIT)/2)))
    else: #test dataset
        data_split = range(int(len(all_label_files)*(TRAIN_SPLIT+(1-TRAIN_SPLIT)/2)),int(len(all_label_files)))
label_files = list(itemgetter(*data_split)(all_label_files))

In [3]:
def blockshaped(arr, nrows, ncols):
    """
    Return an array of shape (n, nrows, ncols) where
    n * nrows * ncols = arr.size

    If arr is a 2D array, the returned array should look like n subblocks with
    each subblock preserving the "physical" layout of arr.
    """
    h, w = arr.shape
    assert h % nrows == 0, f"{h} rows is not evenly divisible by {nrows}"
    assert w % ncols == 0, f"{w} cols is not evenly divisible by {ncols}"
    return (arr.reshape(h//nrows, nrows, -1, ncols)
               .swapaxes(1,2)
               .reshape(-1, nrows, ncols))
def unblockshaped(arr, h, w):
    """
    Return an array of shape (h, w) where
    h * w = arr.size

    If arr is of shape (n, nrows, ncols), n sublocks of shape (nrows, ncols),
    then the returned array preserves the "physical" layout of the sublocks.
    """
    n, nrows, ncols = arr.shape
    return (arr.reshape(h//nrows, -1, nrows, ncols)
               .swapaxes(1,2)
               .reshape(h, w))

In [4]:
def get_row_compressor(old_dimension, new_dimension):
    dim_compressor = np.zeros((new_dimension, old_dimension))
    bin_size = float(old_dimension) / new_dimension
    next_bin_break = bin_size
    which_row = 0
    which_column = 0
    while which_row < dim_compressor.shape[0] and which_column < dim_compressor.shape[1]:
        if round(next_bin_break - which_column, 10) >= 1:
            dim_compressor[which_row, which_column] = 1
            which_column += 1
        elif next_bin_break == which_column:

            which_row += 1
            next_bin_break += bin_size
        else:
            partial_credit = next_bin_break - which_column
            dim_compressor[which_row, which_column] = partial_credit
            which_row += 1
            try:
                dim_compressor[which_row, which_column] = 1 - partial_credit
            except:
                pass
            which_column += 1
            next_bin_break += bin_size
    dim_compressor /= bin_size
    return dim_compressor


def get_column_compressor(old_dimension, new_dimension):
    return get_row_compressor(old_dimension, new_dimension).transpose()

def compress_and_average(array, downsample_factor=10):
    new_shape = (array.shape[0]//downsample_factor, array.shape[1]//downsample_factor)
    # Note: new shape should be smaller in both dimensions than old shape
    newarray = np.mat(get_row_compressor(array.shape[0], new_shape[0])) * \
           np.mat(array) * \
           np.mat(get_column_compressor(array.shape[1], new_shape[1]))
    newarray = np.where(newarray == 0, np.nan, newarray)
    return newarray


def parse_metadata(path_mtl):
  """Returns parsed metadata.

  Args:
    path: path to a local MTL.txt file.

  Raises:
    OSError: if the metadata is missing.

  Returns:
    metadata dict
  """
  if not os.path.exists(path_mtl):
    raise OSError('Missing file: ' + path_mtl)

  metadata = {}
  with open(path_mtl) as f:
    for line in f:
      if ' = ' in line:
        split_line = line.split(' = ')
        try:
          metadata[split_line[0].strip()] = float(split_line[-1].strip())
        except ValueError:
          metadata[split_line[0].strip()] = split_line[-1].strip()
  return metadata


# Constants to use when converting radiance to temperature via the Planck
# equation: https://en.wikipedia.org/wiki/Planck%27s_law
# k1=2hc^2/lambda^5, and k2=hc/lambda k_B, with lambda=wavelength and
# k_B=Boltzmann's constant. Actual satellites also include small calibration
# factors, here called mult and add.
BandConstants = collections.namedtuple('BandConstants',
                                       ['mult', 'add', 'k1', 'k2'])
RADIANCE_ADD_KEY = 'RADIANCE_ADD_BAND_'
RADIANCE_MULT_KEY = 'RADIANCE_MULT_BAND_'
REFLECTANCE_ADD_KEY = 'REFLECTANCE_ADD_BAND_'
REFLECTANCE_MULT_KEY = 'REFLECTANCE_MULT_BAND_'
K1_KEY = 'K1_CONSTANT_BAND_'
K2_KEY = 'K2_CONSTANT_BAND_'


def read_band_constants(metadata,
                        band,
                        reflectance=True):
  """Extract band constants from Landsat Metadata.

  Args:
    metadata: Metadata from the LandSat file.
    band: name of the band to get constants for.
    reflectance: if True, read constants for reflectance rather than
      temperature.

  Returns:
    A BandConstants object containing the constants.
  """
  if reflectance:
    return BandConstants(metadata[REFLECTANCE_MULT_KEY + band],
                         metadata[REFLECTANCE_ADD_KEY + band],
                         None, None)
  else:
    return BandConstants(metadata[RADIANCE_MULT_KEY + band],
                         metadata[RADIANCE_ADD_KEY + band],
                         metadata[K1_KEY + band],
                         metadata[K2_KEY + band])  


def top_of_atmosphere_temperature(image, band_constants):
    """Conversion of radiances to brightness temperature.

    Described here:
    https://www.usgs.gov/core-science-systems/nli/landsat/using-usgs-landsat-level-1-data-product

    Args:
      image: np.ndarray, radiance to convert to brightness temperature.
      band_constants: constants needed by the conversion.

    Returns:
      np.ndarray of brightness temperature (units of K).
    """
    radiance = image * band_constants.mult
    radiance += band_constants.add
    denom = np.log((band_constants.k1 / radiance) + 1.)
    return band_constants.k2 / denom


def reflectance(image, band_constants):
    return image * band_constants.mult + band_constants.add    


def normalize_for_rgb(signal,
                      bounds,
                      adapt=True) -> np.ndarray:
  """Map an array to (0, 1)."""
  start, end = bounds
  out = ((signal - start) / (end - start)).clip(0, 1)
  if (np.all(np.isnan(out) | np.isclose(out, 0, atol=1e-3)) or
      np.all(np.isnan(out) | np.isclose(out, 1, atol=1e-3))):
    adapt = False
  if adapt:
    out = skimage.exposure.equalize_adapthist(out, clip_limit=0.03)
  return out


def get_false_color_image(
    path_11um, path_12um, path_1370nm, path_mtl, night=False):
  """Creates a false color image, as shown to the contrail labelers."""      

  tif_11um = Image.open(path_11um)
  tif_12um = Image.open(path_12um)
  tif_1370nm = Image.open(path_1370nm)

  radiance_11um = compress_and_average(np.asarray(tif_11um))
  radiance_12um = compress_and_average(np.asarray(tif_12um))
  radiance_1370nm = compress_and_average(np.asarray(tif_1370nm))

  del tif_11um
  del tif_12um
  del tif_1370nm

  metadata = parse_metadata(path_mtl)
  constants_11um = read_band_constants(metadata, '10', reflectance=False)
  constants_12um = read_band_constants(metadata, '11', reflectance=False)
  constants_1370nm = read_band_constants(metadata, '9', reflectance=True)  

  temperature_11um = top_of_atmosphere_temperature(radiance_11um, constants_11um)
  temperature_12um = top_of_atmosphere_temperature(radiance_12um, constants_12um)
  reflectance_1370nm = reflectance(radiance_1370nm, constants_1370nm)

  tdiff = temperature_11um - temperature_12um

  # The red band is negative of the brightness temperature difference
  # (so 1 means no brightness temperature difference, 0 means large
  # brightness temperature difference)
  tdiff_clip = (-1, 5.5)  
  tdiff_bounds = (-tdiff_clip[1], -tdiff_clip[0])
  ir_r = normalize_for_rgb(-1 * tdiff, tdiff_bounds, adapt=False)

  # The green band is 1 - cirrus reflectance. So zero means the cirrus band
  # is highly reflective (indicating the presence of cirrus clouds), 1 means
  # the cirrus band is not reflective.
  if night:
    ir_g = np.zeros_like(reflectance_1370nm)
    t12_bounds = (243, 303)  # From a sample of nighttime landsat scenes.
  else:
    ir_g = normalize_for_rgb(1 - reflectance_1370nm, bounds=(0.8, 1))
    t12_bounds = (283, 303)  # From a sample of daytime landsat scenes.

  # The blue band is the 12 um brightness temperature. Zero means cold,
  # (which is consistent with contrails), one means hot, so probably not a
  # contrail.
  ir_b = normalize_for_rgb(temperature_12um, t12_bounds)
  
  return np.stack([ir_r, ir_g, ir_b], axis=-1), temperature_11um, temperature_12um, reflectance_1370nm

def truecolor_rgb(path_red, path_green, path_blue):
  """Given Landsat red, green & blue filepaths, return an image for plotting."""
  
  tif_red = Image.open(path_red)
  tif_green = Image.open(path_green)
  tif_blue = Image.open(path_blue)

  radiance_red = compress_and_average(np.asarray(tif_red))
  radiance_green = compress_and_average(np.asarray(tif_green))
  radiance_blue = compress_and_average(np.asarray(tif_blue))

  del tif_red
  del tif_green
  del tif_blue

  rgb = np.stack([radiance_red, radiance_green, radiance_blue], axis=-1)
  # Landsat gives us the radiance as a 16-bit integer, convert to
  # float in (0, 1).
  rgb = rgb / (2**16 - 1)
  # This routine increases the contrast on dark/bright parts of the image.
  for i in range(3):
    rgb[..., i] = skimage.exposure.equalize_adapthist(
        rgb[..., i], clip_limit=0.1)
  return rgb
#false_color_image, temperature_11um, temperature_12um, reflectance_1370nm = get_false_color_image(path_11um, path_12um, path_1370nm, path_mtl)

In [5]:
def augment_images(mask_slices, img_slices, indexContrail):
    mask_slices = np.expand_dims(mask_slices, axis=3)
    case = np.concatenate((img_slices, mask_slices), axis=3)

    aug_mask_img = case[indexContrail,:,:,:]
    for i in range(7):
        if i == 3:
            aug_mask_img = tf.image.flip_left_right(aug_mask_img)
        else:
            aug_mask_img = tf.image.rot90(aug_mask_img)
        #concatenate each rotation sequence to array of cases
        if random.randint(1, 7)<AUGMENT_SCALING:
            case = np.concatenate((case, aug_mask_img),axis=0)

    #shuffle case order
    np.random.shuffle(case)
    img_slices = np.float16(case[:,:,:,:-1])
    mask_slices = np.int8(np.rint(case[:,:,:,-1]))
    
    return mask_slices, img_slices

In [6]:
def get_slices(image, img_slice_size):
    for i in range(image.shape[3]):
        img_1slice = blockshaped(np.squeeze(image[:,:,:,i]), img_slice_size, img_slice_size)

        if i == 0:
            img_slices = img_1slice
        elif i == 1:
            img_slices = np.stack((img_slices, img_1slice), axis=3)
        else:
            img_slices = np.concatenate((img_slices, img_1slice[:,:,:,np.newaxis]),axis=-1)
           
    return img_slices

In [7]:
def trim_slices(mask_slices, img_slices, indicies, remove_num):
    n_slices = mask_slices.shape[0]
    random_indices = np.random.choice(indicies, size=remove_num, replace=False) 
    mask_slices = np.delete(mask_slices, np.arange(n_slices)[random_indices], 0)
    img_slices = np.delete(img_slices, np.arange(n_slices)[random_indices], 0)
    return mask_slices, img_slices

In [8]:
fileList=tf.io.gfile.glob([save_TFrecord_path + '*' + DVERSION +'.tfrecords']) #existing files

CntNoContrail=0
CntContrail=0

for f in label_files:
    with open (f , "r") as f:
        print(f)
        data = [json.loads(val) for val in f.readlines()]
        for file in data:
            filename_11um = file['filename']
            filename_12um = filename_11um.replace('B10.TIF', 'B11.TIF')
            filename_mtl = filename_11um.replace('B10.TIF', 'MTL.txt')
            TF_record_filename = filename_11um.replace('_B10.TIF', '')+DVERSION
            if save_TFrecord_path+'1_1' + TF_record_filename + '.tfrecords' in fileList:
                continue
            path_11um = os.path.join(image_path, filename_11um)
            path_12um = os.path.join(image_path, filename_12um)
            path_mtl = os.path.join(image_path, filename_mtl)
            
            tif_11um = Image.open(path_11um)
            tif_12um = Image.open(path_12um)
            
            radiance_11um = compress_and_average(np.asarray(tif_11um))
            radiance_12um = compress_and_average(np.asarray(tif_12um))
            
            del tif_11um
            del tif_12um
            
            metadata = parse_metadata(path_mtl)
            constants_11um = read_band_constants(metadata, '10', reflectance=False)
            constants_12um = read_band_constants(metadata, '11', reflectance=False)

            temperature_11um = np.asarray(top_of_atmosphere_temperature(radiance_11um, constants_11um))
            temperature_12um = np.asarray(top_of_atmosphere_temperature(radiance_12um, constants_12um))
            valid_channel = np.full_like(temperature_11um, 0)
            
            filename_1370nm = filename_11um.replace('B10.TIF', 'B9.TIF')
            path_1370nm = os.path.join(image_path, filename_1370nm)
            tif_1370nm = Image.open(path_1370nm)
            radiance_1370nm =  compress_and_average(np.asarray(tif_1370nm))
            del tif_1370nm
            constants_1370nm = read_band_constants(metadata, '9', reflectance=True)
            reflectance_1370nm = reflectance(radiance_1370nm, constants_1370nm)
            #reflectance_1370nm_rng = np.float64([-1, 1]) #good range?
            #reflectance_1370nm = (reflectance_1370nm-np.min(reflectance_1370nm_rng)) / ((reflectance_1370nm_rng[1]-reflectance_1370nm_rng[0]))
            reflectance_1370nm[np.isnan(reflectance_1370nm)] = 0
                
            if NCHANNELS == 7:
                
                filename_red =  filename_11um.replace('B10.TIF', 'B4.TIF')
                filename_green =  filename_11um.replace('B10.TIF', 'B3.TIF')
                filename_blue =  filename_11um.replace('B10.TIF', 'B2.TIF')
                path_red = os.path.join(image_path, filename_red)
                path_green = os.path.join(image_path, filename_green)
                path_blue = os.path.join(image_path, filename_blue)

                rgb = truecolor_rgb(path_red, path_green, path_blue)
                image = np.stack([temperature_11um, temperature_12um, valid_channel, reflectance_1370nm, rgb[:,:,0],rgb[:,:,1],rgb[:,:,2]], axis = 2) 
            else:
                #image = np.stack([temperature_11um, temperature_12um, valid_channel], axis = 2)       
                image = np.stack([temperature_11um, temperature_12um, reflectance_1370nm], axis = 2)  
            
            
            
            # set last channel (valid channel) to 1 for the pixels containing part of the satellite image (keep 0 for the blank pixels)
            #image[image[:,:,0] > 0 ,2] = 1
            
            #make mask from polygons
            mask = np.full_like(temperature_11um, 0)
            imgDim = temperature_11um.shape
            x, y = np.meshgrid(np.arange(imgDim[1]), np.arange(imgDim[0])) # make a canvas with coordinates
            x, y = x.flatten(), y.flatten()
            points = np.vstack((x,y)).T
            
            for polygon in file['polygons']:
                poly = matplotlib.patches.Polygon(np.array(polygon))
                grid = poly.contains_points(points)
                mask = mask+ grid.reshape(imgDim[0],imgDim[1])
           
            #make images and mask all have same h x w (1024,1024)
            tmp = np.zeros((1024,1024,image.shape[2]), dtype = float)
            tmp[0:image.shape[0], 0:image.shape[1], :] += image
            image = tmp
            tmp = np.zeros((1024,1024), dtype = float)
            tmp[0:mask.shape[0], 0:mask.shape[1]] += mask
            mask = tmp
            
            #fig = plt.figure()
            #plt.imshow(mask, cmap = 'gray')
            #plt.show()
            
            #add missing dimensions
            image = np.expand_dims(image, axis=0)
            mask = np.expand_dims(mask, axis=0)

            #normalize img based on channel ranges and set mask            
            BT_rng = np.float64([200, 325]) #good range?
            image[:,:,:,0:2] = (image[:,:,:,0:2]-np.min(BT_rng)) / ((BT_rng[1]-BT_rng[0]))        
                
            #print(f'nans = {np.count_nonzero(~np.isnan(image[:,:,:,:-1]))}')
            image[np.isnan(image)] = 0
            image[image<0] = 0
            image[image>1] = 1

            
            #set true labels = 1
            mask[mask<=1]=0
            mask[mask>1]=1
            
            #fig = plt.figure()
            #plt.imshow(np.squeeze(mask), cmap = 'gray')
            #plt.show()

            #change type to decrease size
            image = np.float16(image)
            mask = np.int8(mask)


            #create matrix with slices of granule of size "img_slice_size"
            mask_slices = blockshaped(np.squeeze(mask), img_slice_size, img_slice_size)
            img_slices = get_slices(image, img_slice_size)
            
            #augment with shifted slices
            if AUGMENT and AUGMENT_SHIFT:
                mask_slices_shift = blockshaped(np.squeeze(mask[:,int(img_slice_size/2):int(-img_slice_size/2),int(img_slice_size/2):int(-img_slice_size/2)]), img_slice_size, img_slice_size)
                mask_slices = np.concatenate((mask_slices, mask_slices_shift),axis=0)
                img_slices_shift = get_slices(image[:,int(img_slice_size/2):int(-img_slice_size/2),int(img_slice_size/2):int(-img_slice_size/2),:], img_slice_size)
                img_slices = np.concatenate((img_slices, img_slices_shift),axis=0)

            n_slices = mask_slices.shape[0]
            ContrailPixelCnt=np.sum(np.sum(mask_slices,axis=1),axis=1) #number of contrail pixels in each mask slice
            
                        
            #balance data set by removing most cases where pixel_threshold is not met 
            if BALANCE == True:
                RatioCorrection = int(np.floor((contrail_ratio/AUGMENT_SCALING)*CntContrail-CntNoContrail))# number of non-contrail slices needed to correct ratio
                indexContrail = np.argwhere(ContrailPixelCnt > pixel_threshold)
                if len(indexContrail)>0:
                    KeepSlices = int(np.floor(len(indexContrail)*(contrail_ratio+1) + RatioCorrection))
                    if len(indexContrail)*(contrail_ratio+1) + RatioCorrection < n_slices: 
                        remove_num = n_slices-KeepSlices
                    else:
                        remove_num = 0
                    indexnocontrail = np.delete(range(n_slices), indexContrail, 0) #array of slice index with no contrail
                    mask_slices, img_slices = trim_slices(mask_slices, img_slices, indexnocontrail, remove_num)
                    
                elif RatioCorrection>=1:
                    if  RatioCorrection < n_slices:
                        remove_num = n_slices- RatioCorrection
                    else:
                        remove_num = 0
                    mask_slices, img_slices = trim_slices(mask_slices, img_slices, range(n_slices), remove_num)
                else:
                    continue
                    
                CntContrail += len(indexContrail)*8
                CntNoContrail += mask_slices.shape[0]-len(indexContrail)
                print(f'ratio={CntNoContrail/CntContrail}')
            
            ContrailPixelCnt=np.sum(np.sum(mask_slices,axis=1),axis=1)
            indexContrail = np.argwhere(ContrailPixelCnt > pixel_threshold)
            if len(indexContrail)>1:
                indexContrail=np.squeeze(indexContrail)
            elif len(indexContrail)>1 == 1:
                indexContrail=indexContrail[0]
            else:
                indexContrail = []
            
            # augment
            if AUGMENT == True and len(indexContrail)>0:
                mask_slices, img_slices = augment_images(mask_slices, img_slices, indexContrail)
            
            max_files = 2048
            write_images_to_tfr_long(img_slices, mask_slices, filename=TF_record_filename, max_files=max_files, out_dir=save_TFrecord_path)



    

<_io.TextIOWrapper name='/home/kjsanche/Desktop/ExternalSSD/SatContrailData/landsat8/labels/landsat_contrails.json-00000-of-00100' mode='r' encoding='UTF-8'>
ratio=11.9


2022-07-13 12:11:45.496909: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-07-13 12:11:45.497862: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-07-13 12:11:45.525751: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:21:00.0 name: Quadro P2000 computeCapability: 6.1
coreClock: 1.4805GHz coreCount: 8 deviceMemorySize: 4.93GiB deviceMemoryBandwidth: 130.53GiB/s
2022-07-13 12:11:45.525808: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2022-07-13 12:11:45.527401: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2022-07-13 12:11:45.527493: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublasLt.so.10
2022-07-1


Using 1 shard(s) for 516 files, with up to 2048 samples per shard


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 12.78it/s]



Wrote 516 elements to TFRecord
ratio=23.925

Using 1 shard(s) for 481 files, with up to 2048 samples per shard


...

100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 15.69it/s]


Wrote 481 elements to TFRecord


ratio=3.7653812865717136

Using 1 shard(s) for 586 files, with up to 2048 samples per shard


100%|█████████████████████████████████████████████| 1/1 [00:00<00:00, 11.24it/s]


Wrote 586 elements to TFRecord


In [9]:
print(f'total slices = {CntContrail+CntNoContrail}, batches = {(CntContrail+CntNoContrail)/64}')


total slices = 2026202, batches = 31659.40625


In [ ]:
os._exit(00)

In [ ]:
#TF_record_filename = filename_11um.replace('v10.TIF.tfrecords', 'v02.tfrecords')
file02 = save_TFrecord_path+'1_1LC08_L1TP_036026_20180217_20180307_01_T1v02.tfrecords'
print(TF_record_filename)
AUTO = tf.data.AUTOTUNE # used in tf.data.Dataset API
BATCH_SIZE=1
data02 = get_training_dataset(file02)

In [ ]:
print(TF_record_filename)

In [ ]:

for x in data02:
    mask = x[1].numpy()
    data = x[0].numpy()
    mslice = mask[0,:,:]
    dslice = data[0,:,:,1]
    print(dslice.shape)
    print(f'min={min(np.min(dslice,0))} max={max(np.max(dslice,0))}')
    print(dslice)
    plt.imshow(np.float32(dslice), cmap = 'gray')
    plt.show()
    plt.imshow(np.float32(mslice), cmap = 'gray')
    plt.show()
    break

In [ ]:

#for i in range(64):
i=11
print(i)

mslizenew=mask_slices[i,:,:]
dslizenew=img_slices[i,:,:,0]
BT_rng = np.float64([200, 330]) #good range?
#image[np.isnan(image)] = 170
dslizenew = (dslizenew-np.nanmin(BT_rng)) / ((BT_rng[1]-BT_rng[0]))
print(f'min={min(np.min(dslizenew,0))} max={max(np.max(dslizenew,0))}')
print(img_slices.shape)
print(dslizenew)
plt.imshow(np.float32(dslizenew), cmap = 'gray')
plt.show()
plt.imshow(np.float32(mslizenew), cmap = 'gray')
plt.show()

In [ ]:
def parse_tfr_element(element):
    
    data = {
      'height': tf.io.FixedLenFeature([], tf.int64),
      'width':tf.io.FixedLenFeature([], tf.int64),
      'depth':tf.io.FixedLenFeature([], tf.int64),
      'raw_label':tf.io.FixedLenFeature([], tf.string),#tf.string = bytestring (not text string)
      'raw_image' : tf.io.FixedLenFeature([], tf.string),#tf.string = bytestring (not text string)
    }


    content = tf.io.parse_single_example(element, data)

    height = content['height']
    width = content['width']
    depth = content['depth']
    raw_label = content['raw_label']
    raw_image = content['raw_image']


    #get our 'feature'-- our image -- and reshape it appropriately
    feature = tf.io.parse_tensor(raw_image, out_type=tf.float16)
    feature = tf.reshape(feature, shape=[height,width,depth])
    label = tf.io.parse_tensor(raw_label, out_type=tf.int8)
    label = tf.reshape(label, shape=[height,width])
    print(feature[1,1,1])
    return (feature, label)

def get_batched_dataset(filenames, testing = False):
    option_no_order = tf.data.Options()
    if testing:
        option_no_order.experimental_deterministic = True
    else:
        option_no_order.experimental_deterministic = False

    #dataset = tf.data.TFRecordDataset(filenames)
    dataset = tf.data.Dataset.list_files(filenames)
    dataset = dataset.with_options(option_no_order)
    dataset = dataset.interleave(tf.data.TFRecordDataset, num_parallel_calls=AUTO)
    dataset = dataset.map(parse_tfr_element, num_parallel_calls=AUTO)

    dataset = dataset.cache() # If dataset fits in RAM
    if not testing:
        dataset = dataset.shuffle(2048)
    #dataset = dataset.repeat()
    dataset = dataset.batch(BATCH_SIZE, drop_remainder=True) 
    dataset = dataset.prefetch(AUTO) #

    return dataset


def get_training_dataset(training_filenames):
    return get_batched_dataset(training_filenames)

In [ ]:
T11min, T11max, T12min, T12max = 300, 300, 300, 300
dim1min, dim1max, dim2min, dim2max = 780, 780, 780, 780
for f in label_files[3:]:
    print(f'T11={T11min}-{T11max}, T12={T12min}-{T12max}, dim1={dim1min}-{dim1max}, dim2={dim2min}-{dim2max}')
    with open (f , "r") as f:
        data = [json.loads(val) for val in f.readlines()]
        for line in data:
            filename_11um = line['filename']
            filename_12um = filename_11um.replace('B10.TIF', 'B11.TIF')
            filename_mtl = filename_11um.replace('B10.TIF', 'MTL.txt')
            
            path_11um = os.path.join(image_path, filename_11um)
            path_12um = os.path.join(image_path, filename_12um)
            path_mtl = os.path.join(image_path, filename_mtl)
            
            tif_11um = Image.open(path_11um)
            tif_12um = Image.open(path_12um)
            radiance_11um = compress_and_average(np.asarray(tif_11um))
            radiance_12um = compress_and_average(np.asarray(tif_12um))
            
            del tif_11um
            del tif_12um
            
            metadata = parse_metadata(path_mtl)
            constants_11um = read_band_constants(metadata, '10', reflectance=False)
            constants_12um = read_band_constants(metadata, '11', reflectance=False)

            temperature_11um = np.asarray(top_of_atmosphere_temperature(radiance_11um, constants_11um))
            temperature_12um = np.asarray(top_of_atmosphere_temperature(radiance_12um, constants_12um))
            image = np.stack([temperature_11um, temperature_12um], axis = 2)
            
            imgdim = temperature_11um.shape
            dim1min, dim1max = np.min([dim1min, imgdim[0]]), np.max([dim1max, imgdim[0]])
            dim2min, dim2max = np.min([dim2min, imgdim[1]]), np.max([dim2max, imgdim[1]])           
            
            T11min, T11max = np.nanmin([T11min,  np.nanmin(temperature_11um)]), np.nanmax([T11max,  np.nanmax(temperature_11um)])
            T12min, T12max = np.nanmin([T12min,  np.nanmin(temperature_12um)]), np.nanmax([T12max,  np.nanmax(temperature_12um)])



    

In [ ]:
#after looking at first 40 of 100: T11=197.8631130400431-325.5216346939916, T12=204.3156141767293-324.2603994258813, dim1=646-916, dim2=654-917